## Imports

In [2]:
import requests
from azure.storage.blob import ContainerClient

## File download function

In [3]:
# downloads a file based on url and saves it with the specified file path and name.

def downloadFile(filename, download_url):   
    req = requests.get(download_url)
    with open(filename, 'wb') as file:
        file.write(req.content)  

## Azure Blob Storage upload function

In [5]:
# upload file to azure blob storage container

def uploadAzureBlob(input_file_path, output_blob_name, container_name='processed'):
    connection_string = "DefaultEndpointsProtocol=https;AccountName=lirkovblob;AccountKey=j7S+VWpwAGd4dWfHudiMXu18K9QiY6x61mXUN16qEjxbyT4L0qy9PsW4F7NogGjDHf6v/iUqS512DQnMjNTfYw==;EndpointSuffix=core.windows.net"
    # client to interact with a specific container
    container_client = ContainerClient.from_connection_string(conn_str=connection_string, container_name=container_name)
    # upload file
    with open(input_file_path, "rb") as data:
        container_client.upload_blob(name=output_blob_name, data=data)
    print(f'File "{output_blob_name}" has been uploaded to "{container_name}" container.')

## Data transformation functions

In [4]:
# clean column names
def fixColumnNames(input_df):
    columns = []
    for column in input_df.columns:
        column = column.title()
        column = column.replace(" " , "")
        column = column.replace("-" , "")
        column = column.replace("_" , "")
        column = column.replace("," , "")
        column = column.replace(";" , "")
        column = column.replace("{" , "")
        column = column.replace("}" , "")  
        column = column.replace("(" , "")
        column = column.replace(")" , "")
        column = column.replace("\t", "")
        column = column.replace("=" , "")
        columns.append(column)
    return columns

# reorder columns based on index
def reindexCols(input_df, order_list):
    if isinstance(order_list, list) == True:
        cols_list = list(input_df.columns)
        cols_order = order_list
        cols_ordered = [cols_list[i] for i in cols_order]
        input_df = input_df.reindex(columns=cols_ordered)
        return input_df
    else:
        return 'list of columns has not been provided'

# create a list with column names of specific data type
def colTypesList(input_df, dtype):
    int_cols = input_df.dtypes.index[input_df.dtypes.values == dtype].to_list()
    return int_cols

# check the max length of each column
def checkColumnLen(input_df):
    length_dict = {}
    for column in input_df.columns:
        length_dict[column] = [max(input_df[column].astype(str).apply(len)), min(input_df[column].astype(str).apply(len))]
    return length_dict

# change data type
def changeDataType(input_df, column, data_type):
    input_df[column] = input_df[column].astype(data_type)